In [12]:
f = open("bloom-syndrome-protein.txt", "r")
lines = f.readlines()
genome = ''.join(''.join(lines).split())
genome

'gaagatgctcaggaaagtgactctctgaaaactcatttggaagatgaaagagataatagcgaaaagaagaagaatttggaagaagctgaattacattcaactgagaaagttccatgtattgaatttgatgatgatgattatgatacggattttgttccaccttctccagaagaaattatttctgcttcttcttcctcttcaaaatgccttagtacgttaaaggaccttgacacctctgacagaaaagaggatgttcttagcacatcaaaagatcttttgtcaaaacctgagaaaatgagtatgcaggagctgaatccagaaaccagcacagactgtgacgctagacagataagtttacagcagcagcttattcatgtgatggagcacatctgtaaattaattgatactattcctgatgataaactgaaacctttggattgtgggaacgaactgcttcagcagcggaacataagaaggaaacttctaacggaagtagattttaataaaagtgatgccagtcttcttggctcattgtggagatacaggcctgattcacttgatggccctatggagggtgattcctgccctacagggaattctatgaaggagttaaatttttcacaccttccctcaaattctgtttctcctggggactgtttactgactaccaccctaggaaagacaggattctctgccaccaggaagaatctttttgaaaggcctttattcaatacccatttacagaagtcctttgtaagtagcaactgggctgaaacaccaagactaggaaaaaaaaatgaaagctcttatttcccaggaaatgttctcacaagcactgctgtgaaagatcagaataaacatactgcttcaataaatgacttagaaagagaaacccaaccttcctatgatattgataattttgacatagatgactttgatgatgatgatgactgggaagacataatgcataatttagcagccagcaaatcttccacagct

In [27]:
# Basic Run length encoding 
MAX_RUN = 4 #not being used currently
genome_size = len(genome)
compressed = ""
i = 0
while i < genome_size: 
    run_length = 1
    k = i+1
    while k < genome_size and genome[i]==genome[k]:
        run_length += 1
        k += 1
   
    compressed += genome[i] + str(run_length)*( 0 if run_length==1 else 1)
    i = k   
len(compressed)

3139

In [29]:
# Gzip compression
import gzip
compressed_gzip = gzip.compress(bytes(genome, 'utf-8'))
print (len(compressed_gzip), compressed_gzip)

1120 b'\x1f\x8b\x08\x00yI\x81a\x02\xffU\x97Q\x96[1\x08C\xd7\xca\xe1\x83\r\xb0\xffSKW\xbct\xdat\x9a8~6HB0S5\xb5\xd3\xdb5S\xef\xc3N\xbd\x0f\xef\xaf>\xe9m\xed\xae\xbe\xd26m\xd0\x9bz\xafi\xef\x98{}\xdb\xde7\xeb\xcf\xa5g[\x87\xe8!\x9d\xbd\xfd\x96f\xdfr\x1e\xa8\xdfk\xfd\xb3z\xf4\xfe}\xe5\xcd\xdd\xab`:wx\x97V\xc6\xeb~\xb5\x7f*\x94\xb7\xfa\xde\xbek\xde!\xef\xffRFZy?\xabI\xc9\x079\t]\xea\x03^\xb8\n\xb4Y\xd7\x8a.\xf7\xc7\xbe\xc8\x15\x9a\xc2\x1e\xa3\x94\xfc\x88\xaa\xf4_\xfb\xe0\xb7\xac+\xde\xd7\xc5M\xeb\x9c\x85\xc3\xe4\xe5\xf8\x8d\x81\xee\x9f\xbb[\x9889#\xf3\x1e\xf0\xb6\x0e2\x05\x84\x8e\xc7 +\xc0\x11\xd8\xa2 `p\xbe\x17}o\xc1\xa7A*\xa3\xaa\x0c\xc0\xb6|f\x11\x84\xe2\x0f\x963\xf0=\x0e\xad\x1c\xf8\x10\xc6[o\xf8z\x0b\nO;f\xf8J!hQ\xdbM\xac78\x02\x88\xd0\xa5:\x01>\xc5E\xb1m\x16~\x95\x0e\x00\n.k\xb0L\xbf\xb0Dv\x86~\x91\xa6\xbf\xaa\xc8\r\xca\x10\xa7\x15\x00\xc6\n\xbf\xad^X\x9f\x05\xbe\x05\x08t9\x13du\x9e)\xe4:\xfep\xa6\x05f\xd9\xe5\xce(G\xf9\x98@\x110)\x8e\xf5]\xa6\xcc<\xc2\x8d\x17\x9c\xd5\xd6\x15Q9\xbc \

In [30]:
# Bzip2 compression
import bz2
compressed_gzip = bz2.compress(genome)
print (len(compressed_gzip), compressed_gzip)

1020 b'BZh91AY&SY\xa1\x9b\x07\xcf\x00\x03\xbe\xc1\x80z\x00(\x80\x04\x00`\x07\xdd\xf0\xbcz\xe77U\xbb\xba\xd3\xbai\xb8\xcbn\xdd\xb5\xdd\xda\xed\x9c\x10d\xc4\rO\xc5*\x8c\r<T\x94\x02T\xfc\x92\x956\xa1\x06L@\x94\xf2RJ7\xf1\xf9\x1f\xd4\x9f\x82DE\x17\xc3\xa8TD\xcf\x88\x159\x13\xa7\x83\xbb\xa0\xc6\x99\xe4J!E\x07\xed\xb3\x15\xd2\x12U<\x13S\xc9\xf3\xd9\xb6\xd3T\x8bt|&\x940\xad\x865Tk3T\xf3\xa2\xd1\xbb1;\xb1\x195\x1a\xb1Y\x04\x8aY&-\x11\x87\xce\xae\xb8]\xf3\xeb\xe8\xe1\xd8\xad\xb3\x11P\x8b\xa4\x92\xe7\x82G\n\xc4\xa6\xb1\xc9Y\xd0\xd8\x16\x025\xda\xccN\x13\x1a1T\xd7\x17\xcf\xb1\xe5\xc3\xe6\xa1W6e\xb3\x01h\x9c\xbc\xeb%\xa6\x8c\xf4\xf5\x8b\x13\x90X\x05,\x11s\xc7\xad\xcc\x85g\x0e4\xb8\xb0\x81\xad\xea\xb0q\xc1\xceS\xd5tdr\x99D\xe1sd\x87\x16\xa2n\xd0ddGn\xdd\x90Ts\x913fz\x88\xaajvv\xd0\xc80\xb3SS+\xcep\xf6,\x8cwK\xdc\xb4 \xa4\x90I\xb4\x9bI\xa5\xd8\xb48\x19\x872Q\xd2\x83h\x82\x11\xc3\x19\xd5\x94\x9cH\x89\xa8\xe2\x12r\xf5\\\xc8\x9eZs@a\x00\xf3\x9c\x80\xdat\xf6&dV\xc1\x87<\x8a\xa4i\xadZ\n3V\x00\x92\xd4sv\x